# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Project Submission

## Notebook content

- Data Initialization
- General Analysis
- Feature Engineering
- Feature Analysis
- Model Training
- Model Evaluation
- Accuracy Prediction

In [ ]:
import os

import pandas as pd

from data_management import DataManager

## Data Initialization
To ensure that all tasks like the analysis and the feature engineering are conducted on the same, consistent dataset, a custom data manager class has been created. The data manager provides methods, that are needed and can be re-used in multiple tasks (e.g. retrieval of orders with tip information for the training dataset). Since the data management functionality is encapsulated in the data manager, the room for errors caused by redundant implementations is reduced and the code is more maintainable, because methods can be adjusted in a single location.

In [ ]:
DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))

tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))
aisles = pd.read_csv(os.path.join(DATA_DIR, 'aisles.csv.zip'))
departments = pd.read_csv(os.path.join(DATA_DIR, 'departments.csv.zip'))
products = pd.read_csv(os.path.join(DATA_DIR, 'products.csv.zip'))

data_manager = DataManager(op_prior, op_train, tip_train, tip_test, orders, products, aisles, departments)
order_amount = len(data_manager.get_orders_tip())

## Data Analysis
Several analysis were done to find any tip patterns in the data. The purpose of this analysis is to find specific features that are relevant for the prediction of tip. To find out which attributes have a significant influence on the tip, several analysis tasks were executed.

In [ ]:
from analysis import ReorderedAnalysis, ProductCardOrder, GeneralFacts, DayOfWeek, HourOfDay, Department, Aisle, \
    Product, OrderNumber

TODO: Describe here your analysis task. What is the analysis about, any assumptions before? What did we expect?
TODO: Describe the results also here in this Markdown



##### **Analysis Task Name**
**Analysis:** 

**Result Interpretation:**

In [ ]:
# TODO: Initialize your analysis instance
# TODO: Produce your output

##### **General Analysis**
**Analysis:** The general analysis task was created to get an overview of the data set and to find out general facts about the dataset and its data. Typical functions like the mean, median, standard deviation, min and max values were calculated for the different entities. The purpose of this analysis task was to get a first impression of the data set and to find out if there are any irregularities or outliers in the data. The datatypes of the different attributes were checked, and the number of missing values was calculated.

**Result Interpretations:** 
- The dataset mostly contains numerical attributes with a few categorical attributes.
- The dataset is rather large with 32 million rows and 16 columns.
- The dataset contains some missing values in the columns "days_since_prior_order" and some in "tip". The missing "tip" values are limited to the test data.


In [ ]:
general_analysis = GeneralAnalysis(data_manager)
general_analysis.execute_analysis()

##### **General Facts**
**Analysis:** For data understanding and getting familiar with the data set and the overall relations between the main entities, some general facts about the dataset were collected. Therefore several facts has been calculated containing the different entities and there relations.

**Result Interpretations:** The results are important to clarify weather the calculations fit with the overall facts of the dataset and do not contain any calculation errors or changed the dataset.

In [ ]:
general_facts = GeneralFacts(data_manager)
general_facts.execute_analysis()

##### **Day of the Week Analysis**
**Analysis:** The analysis was conducted to find out whether the day of the week has an impact on the tip probability. To find out if there is a connection between the day of the week and the tipping behaviour, 2 plots were created.
- Left plot: The left plot shows the absolute number of orders that were tipped and not tipped depending on the day of the week on which the order was placed as a bar chart with overlapping bars. This was done to get an overview of how the orders are distributed over the days of the week, to put the tip probability into perspective.
- Right plot: The plot on the right shows the probability that an order was tipped depending on the day of the week on which the order was placed. The tip probability was calculated by dividing the number of tipped orders by the total number of orders for each day of the week.

**Result Interpretation:**
The right plot shows that the tip probability is significantly higher on day 0 and 1 (probably Saturday and Sunday) than on the other days of the week. While the tip probability is relatively constant on the other days of the week at or below 40%, the tip probability on day 0 and 1 is above 50%.

&rarr; **Impact of order_dow**: The feature **dow_high_tip_probability**, which one-hot encodes whether the order was placed on the days on saturday/sunday or not, was created to capture this relationship. This was necessary, because the order_dow is an ordinal-scaled variable, which cannot be used directly in most models (e.g. logistic regression).


In [ ]:
day_of_week = DayOfWeek(data_manager)
day_of_week.execute_analysis()

##### **Hour of the Day Analysis**
**Analysis:** The analysis was conducted to find out whether the hour of the day, in which the order has been placed, has an impact on the probability that the order is tipped. To find out if there is a connection, the same plots were created as in the day of the week analysis, only this time with the hour of the day.

**Result Interpretation:**
The right plot shows that the tip probability is significantly higher in the early morning and in the evening than during the day. The period in the early morning, where the tip probability is higher, includes the hours 0-4 (from 12 until 5 a.m.). At hour 5 a significant drop in the tip probability can be observed. The tip probability remains relatively constant during the day with a slight increase in the morning hours. Since the change in the tip probability is not as drastic as in the early morning and doesn't reach the same level as in the early morning or the evening, this increase is not considered significant. At hour 19, the tip probability increases abruptly and remains at a high level until hour 23 (7 p.m. until 12 a.m.). Therefore the hours 0-4 and 19-23 are considered as the hours with a high tip probability.

In the left plot, it can be observed that the number of placed orders is distributed very unevenly over the hours of the day. In the hours with a high tip probability, the number of orders is significantly lower than during the day. This is particularly noticeable in the early morning hours, where the number of orders is very low. Therefore, the significant increase in the tip probability in the early morning hours is not as relevant as the increase in the evening, where the number of orders is higher. Despite this fact, no differentiation was made between the early morning and the evening, for simplicity reasons and since the tip probability is very similar in both periods, which cover a continuous time span (7 p.m. until 5 a.m.) when combined.

&rarr; **Impact of order_dow**: The feature **hod_high_tip_probability**, which one-hot encodes whether the order was placed in the hours with a high tip probability (0-4 and 19-23) or not, was created to capture this relationship. This is necessary because the order_hour_of_day is also an ordinal-scaled variable.

In [ ]:
hour_of_day = HourOfDay(data_manager)
hour_of_day.execute_analysis()

##### **Order Number Analysis**
**Analysis:** The analysis of the order number was conducted to find out whether the attribute has an impact on the tip probability. Again, the same plots were created as in the day of the week analysis, only this time with the order number. To better visualize the relationship, a line graph was chosen to show the tip probability in dependence of the order number, instead of a bar plot.

**Result Interpretation:**
The right plot shows that the probability that an order is tipped increases with a rising order number up until around the 40th order. After that, the tip probability decreases again. In the graph one can clearly see that the curve is not linear, but rather resembles the shape of a quadratic function.

The left plot shows a decreasing number of orders with a rising order number. This was expected due to the fact, that the order number is a user-specific attribute and there are more users with a shorter order history than with a longer order history. Therefore, an order with a larger order number is less likely to appear. This also reduces the significance of the tip probability in the right plot for higher order numbers, because the number of orders is very low. However, this is not encoded explicitly in the feature, but will be left to the model to learn an appropriate weight.

&rarr; **Impact of order_number**: The feature **order_number_squared** was created, which contains the squared order number. This feature was created to allow linear models to capture the quadratic relationship between the order number and the tip probability, when combined with the original order number feature.


In [ ]:
order_number = OrderNumber(data_manager)
order_number.execute_analysis()

##### **Department Analysis**
**Analysis:** The analysis of the department was conducted to find out whether orders containing products from a specific department have a higher tip probability than others. The same 2 plots were used to visualize, on one hand, the absolute number of tipped and not tipped orders containing products from a specific department and on the other hand the tip probability of orders containing products from a specific department.

**Result Interpretation:** The right plot shows that the tip probability of orders containing products from the alcohol department is significantly higher than the tip probability of orders containing products from other departments. Even though the tip probability of orders containing products from the other departments is not constant, the differences between the departments are not as significant as the difference between the alcohol department and the other departments.

The left plot shows that the number of orders containing products from the alcohol department is rather low compared to the other departments. But since the difference in the tip probability is so striking, it was decided to create a feature which encodes this relationship anyway.

&rarr; **Impact of department**: The feature **contains_alcohol** was created, which one-hot encodes whether the order contains at least 1 product from the alcohol department or not. This feature was created to capture the significant difference in the tip probability of orders containing products from the alcohol department compared to orders containing products from other departments, without one-hot-encoding all departments. This would have resulted in 21 additional features (1 for every department), which would have lead to a very sparse feature matrix and would have made the model more complex. Additionally, the feature **dept_tip_rate**, which is described in the feature analysis section, was created to capture the relationship between the tip probability and the departments in a more generic and dense way.

In [ ]:
departments = Department(data_manager)
departments.execute_analysis()

##### **Aisle Analysis**
**Analysis:** The analysis of the aisle was conducted to find out whether orders containing products from a specific aisle have a higher tip probability than others. For this analysis, a single plot was created, which shows the tip probability of orders containing products from a specific aisle. In addition to that, aisles belonging to the alcohol department were highlighted.

**Result Interpretation:** The plot reveals that orders including items from the alcohol aisles have a notably higher likelihood of receiving tips compared to orders that contain items from other aisles. While the tip probability for non-alcohol aisles varies, none exhibit a distinctly higher likelihood of tipping, unlike the alcohol aisles. Generally, there is a mild fluctuation in tip probability across these other aisles, with some slightly higher and others lower, but these variations are not as pronounced as those observed for the alcohol aisles.

&rarr; **Impact of aisle**: The analysis did not reveal any significant relationship between a specific aisle, other than the alcohol aisles, which are already encoded in the **contains_alcohol** feature. Therefore, no feature was created to capture the relationship between the tip probability and a specific aisle. One-hot-encoding all 134 aisles would also not be an option. Instead, **aisles_tip_rate** was created, which is described further in the feature analysis section and encodes the fluctuation of the tip probability across all aisles in a more generic and dense way.

In [ ]:
aisles = Aisle(data_manager)
aisles.execute_analysis()

##### **Product Analysis**
**Analysis:** The analysis of the product was conducted to find out whether specific products have a higher tip probability than others. For this analysis, multiple plots and tables were created to visualize the tip probability of the products:

- First plot: The first plot visualizes the distribution of products depending on the probability, that orders containing these products are tipped. The products are grouped into 100 bins, which means that every bin contains products with a tip probability between 0 and 0.01, 0.01 and 0.02, and so on. The plot shows the number of products in each bin.
- Second plot: The second plot visualizes the same distribution, but this time the products were weighted by how often they were ordered. This was done to judge the significance of the tip probability of the products. The plot shows the number of orders containing products in each bin.

- Third plot: The third plot displays the same distribution of products as the first plot, but in a more fine grained way. Only products with a tip probability between 80% and 100% are shown, grouped into 20 bins.
- Fourth plot: The fourth plot visualizes the same distribution as the third plot, but the products were weighted by how often they were ordered.
- Table 1: The table shows products with a 100% tip probability, that don't belong to the alcohol department, sorted by the number of orders containing these products.

- Fifth plot: The fifth plot is equivalent to the third plot, but only products with a tip probability between 0% and 20% are shown, grouped into 20 bins.
- Sixth plot: The sixth plot is equivalent to the fifth plot, but the products were weighted by how often they were ordered.
- Table 2: The table shows products with a 0% tip probability, sorted by the number of orders containing these products.
- Last plot: The last plot shows the distribution of products with a tip probability of 0% depending on the number of orders containing these products.

In all plots, products from the alcohol department are highlighted.

**Result Interpretation:**
When looking at the first and the second plot, one can see, that the tip probability varies between different products. The distribution of the tip probability of the products is not uniform, but rather resembles a normal distribution, except there are noticeable peaks at 0% and 100%. Once the products are weighted by how often they were ordered, the peaks at 0% and 100% disappear and the distribution becomes more uniform. This leads to the conclusion that these products are not ordered very often and therefore the tip probability is not very significant. Only a small elevation can be observed starting at around 80% tip probability, which indicates that there are some products with a high tip probability, that are ordered more often than others.

In the third and forth plot, this is examined in more detail. The unweighted plot on the left shows that there are many products, which do not belong to the alcohol department and have a tip probability between 80% and 100%. There is also a large peak of products, that are tipped every time they are ordered. However, when looking at the weighted plot on the right, this peak shrinks drastically. It also becomes clear that the alcohol products are extremely dominant in this area and that the other products are not ordered very often, when looking at the weighted plot. This leads to the conclusion that the tip probability of the alcohol products is rather significant, while the tip probability of the other products is not. The table 1 confirms this, as it shows that the products, which do not belong to the alcohol department and have a 100% tip probability, appear in 7 orders at most. When considering that in the dataset as a whole, products are ordered an average of 653 times, with a median order frequency of 60 times, it becomes clear that the non-alcohol products with a 100% tip probability are not very significant. Therefore, a feature using a curated list of products that are tipped every time (or above 80%) would not be very useful, since most of these products are alcohol products or are not ordered very often.

A similar observation can be made for the products with a tip probability between 0% and 20%. The unweighted plot on the left shows that there are many products with a low tip probability with a significant peak at 0%. But when looking at the weighted plot on the right, it becomes clear that these products are also not ordered very often, as the peak at 0% shrinks drastically. Table 2 shows, that the products with a 0% tip probability are ordered 22 times at most, which is more significant than the products with a 100% tip probability, but still low. The last plot gives a more detailed view of the distribution of the products with a 0% tip probability. It shows that the products with a 0% tip probability are ordered very rarely, with the majority of them being ordered only 3 times. Not many of these products are ordered more than 10 times. Therefore, a feature using a curated list of products that are never tipped would also not be very useful.

&rarr; **Impact of product**: The analysis showed that products that are always/never tipped are not very significant, since they are not ordered very often. Therefore, no feature was created to capture the relationship between the tip probability and a curated list of products that are always/never tipped. To allow the model to utilize the information that some products are more likely to be tipped than others, which is clearly visible in the first/second plot, the generic feature **product_tip_rate** was created, which is described further in the feature analysis section.

In [ ]:
products = Product(data_manager)
products.execute_analysis()

##### **Add to card order analysis per Product**
**Analysis:** To find out whether the add to card order of a specific product has an impact on the tip probability of that product, an analysis task has been created which calculates the tip probability of the products in dependence of the add_to_cart_order. To compare the "add to card order" of a product, the rational scaled variable (add_to_card_order) has to be normalized. With that normalization every "add to cart order" is transformed to an additional normalized_add_to_card attribute order within the interval [0,1]. After the calculation of the normalized attribute, the variable has been subdivided in 10 bins. Within these bins the mean tip probability of every product is calculated and visualized within the box plot below.

**Result Interpretations:** The boxplot below shows the distribution of the product specific tip probabilities in dependence of the normalized "add to card order". It displays the median, potential outliers and the area between the 25th and the 75th quartile which represents 50% of the data. Considering the area between the 25th and 75th quartile, it can be recognized that there is no overall impact of the normalized add_to_card oder on the product specific tip probability. Focussing the outliers, it can be seen that there are some differences between the different bins. However, because the range between the 25th and 75th percentile is so constant, it can be assumed that these outliers are not necessarily related to "add to card or" but are due to other influences.\
&rarr; **No overall impact of add_to_card_order on the tip probability of the products** &rarr; no feature crafted and no further analysis on the attribute "add_to_card_order"

In [ ]:
product_card_order = ProductCardOrder(data_manager)
#product_card_order.execute_analysis()

##### **Reordered tip probability analysis per Product**
**Analysis:** To find out weather the amount of reorders has an influence on the tip probability of a product, an analysis has been done which compares the product specific tip probability in dependence of the product reordered probability. Because of the fact that the "reordered" attribute is a user specific flag, the calculated product reordered rate describes the probability that a product is reordered. With a high reordered rate we can assume that the product has been bought quite often across all users. The heatmap below shows the product tip probability in comparison to the reordered rate. The purpose of this analysis task was to identify weather the reordered rate has an overall influence of the tip probability. The assumption here was that the tip probability of the products generally increases with a rising reordered rate.

**Result Interpretations:** The heatmap overall shows that there is no general connection between the reordered rate and the product specific tip probability. As the reordered rate increases, the product tip probability of the products remains relatively constant and does not change depending on the reordered rate.\
&rarr; **No overall impact of reordered rate on the tip probability of the products** &rarr; no feature crafted

In [ ]:
reordered_tip_rate = ReorderedAnalysis(data_manager)
reordered_tip_rate.execute_analysis()

##### **Number of Orders per User**
**Analysis:** To find out weather the number of orders per user has an influence on the tip probability, an analysis has been done which compares the tip probability in dependence on the number of orders per user. The purpose of this analysis task was to identify weather the number of orders per user has an overall influence on the tip probability. 
For the analysis, the number of orders per user has been calculated and the tip probability has been calculated for every user. The tip probability has been calculated by dividing the number of tipped orders by the number of orders of the user. The tip probability has been calculated for every user and the mean tip probability has been calculated for every number of orders per user. The results are visualized in the line plot below.

**Result Interpretations:** The line plot below shows that the tip probability of the users increases constantly with the number of orders per user until it reaches a ceiling at around 50 orders per user. Afther that the tip propability starts a slight downward trend which is rather unstable. To note is that the most data points are located in the range of 0 to 50 orders per user.
This analysis shows that the number of orders per user has a significant impact on the tip probability and should be considered as a feature.

In [ ]:
num_order_user = NumberOrderUser(data_manager)
num_order_user.execute_analysis()

## Feature Engineering
With the results of the general analysis task and our specific assumptions of any relation between tip and various data combinations, a few features were build that we think will make a potential impact. The explanation of each feature and corresponding impact analysis will be covered in the next chapter (Feature analysis). To execute the impact analysis of the features, it is necessary to compute them all before.

For the computation of the features, it is distinguished between two types of features:
- **Static features:** Features that are calculated user specific and consider just the last orders of the specific user. These features can be computed once and do not need to be recalculated for a newly created fold in a time series cross validation.
- **Dynamic features:** Features that are calculated user comprehensive like the product tip rate of all orders. For these features we use to data of the whole dataset or fold in cross validation. Therefore the dynamic features need to be recalculated for every new fold in the cross validation.

In [ ]:
from feature_engineering.static_features import ContainsAlcohol, DowHighTipProbability, TipHistory, OrderSize, \
    OrderNumberSquared, PrevOrderTipped, MeanOrderedRate, CustomerLifetime, ReorderedRatio, OrderFrequency, \
    HodHighTipProbability, PrevTippedProductsRatio, RelDaysSinceTip, DaysSinceTip, LastTipSequence, AvgSizePrevOrders, \
    SimOrdersTipRatio
from feature_engineering.dynamic_features import ProductTipRate, DepartmentTipRate, AisleTipRate, AssocRulesAisles, \
    AssocRulesDepartments

In [15]:
# Static Features
tip_history = TipHistory()
reordered_rate = ReorderedRatio()
order_size = OrderSize()
prev_tipped_products_ratio = PrevTippedProductsRatio()
customer_lifetime = CustomerLifetime()
prev_order_tipped = PrevOrderTipped()
order_frequency = OrderFrequency()
mean_ordered_rate = MeanOrderedRate()
rel_days_since_tip = RelDaysSinceTip()
days_since_tip = DaysSinceTip()
sim_orders_tip_ratio = SimOrdersTipRatio()
product_tip_rate = ProductTipRate()
order_number_squared = OrderNumberSquared()
hod_high_tip_probability = HodHighTipProbability()
dow_high_tip_probability = DowHighTipProbability()
contains_alcohol = ContainsAlcohol()
avg_size_prev_orders = AvgSizePrevOrders()

# Dynamic Features
department_tip_rate = DepartmentTipRate()
aisle_tip_rate = AisleTipRate()
last_tip_sequence = LastTipSequence()
assoc_rules_departments = AssocRulesDepartments()
assoc_rules_aisles = AssocRulesAisles()

In [16]:
# Static Features
data_manager.register_feature(tip_history)
data_manager.register_feature(reordered_rate)
data_manager.register_feature(order_size)
data_manager.register_feature(customer_lifetime)
data_manager.register_feature(prev_order_tipped)
data_manager.register_feature(prev_tipped_products_ratio)
data_manager.register_feature(order_frequency)
data_manager.register_feature(sim_orders_tip_ratio)
data_manager.register_feature(mean_ordered_rate)
data_manager.register_feature(last_tip_sequence)
data_manager.register_feature(rel_days_since_tip)
data_manager.register_feature(days_since_tip)
data_manager.register_feature(order_number_squared)
data_manager.register_feature(hod_high_tip_probability)
data_manager.register_feature(dow_high_tip_probability)
data_manager.register_feature(contains_alcohol)
data_manager.register_feature(avg_size_prev_orders)

# Dynamic Features
data_manager.register_feature(product_tip_rate)
data_manager.register_feature(department_tip_rate)
data_manager.register_feature(aisle_tip_rate)
data_manager.register_feature(assoc_rules_aisles)
data_manager.register_feature(assoc_rules_departments)

Because of the computing time of each feature, the calculation has been done once before and is imported to the local file system. If it is required to compute the features once again, the next two lines of code need to be executed.

In [17]:
# In case a recomputing of the features is necessary. Remove the comments for the next two lines of code.
# data_manager.compute_features()
# data_manager.export_features('data/prepared_data/computed_features.csv.zip', only_static=False)

data_manager.import_features('data/prepared_data/computed_features.csv.zip', only_static=False)

## Feature analysis
To decide weather a feature will take place in the feature set of the machine learning model, it is necessary to analyze which features have a significant impact on tip. All features that has been computed are described and analyzed in the following section.


In [ ]:
# Dataset for analysis
orders_tip_features = data_manager.get_orders_tip_train()

##### **Feature_Name**
**Description:** Describe how the feature is calculated and what does it mean.

**Analysis:** Which impact does the feature have on tip. Does it have any correlations to other features
&rarr; **Is the feature part of the model feature set? and why (one sentence)**

In [18]:
# TODO: Execute the feature_name.analyze_feature() method

##### **Average size of the previous orders(avg_size_prev_orders)**
**Description:** The feature represents the average size of the previous orders of the user.
For this, the size of the previous orders of the user is calculated and the mean of the sizes is taken.

**Analysis:** The feature has a correlation which is near zero. Furthermore, after the analysis of the feature, it can be seen that the feature has no significant impact on the tip.
The consequence of the low correlation and analysis is that the feature is not part of the model feature set.

In [ ]:
avg_size_prev_orders.analyze_feature(orders_tip_features)

##### **Last Tip Sequence**
**Description:** The feature represents the sum of the length of the last tip sequence. In case the last order has not been tipped, the feature value is always zero. In case the last order has been tipped, all directly consecutive tipped orders are added to the sequence. A "last_tip_sequence" of five means that the last five orders had all been tipped by the user.

**Analysis:** Which impact does the feature have on tip. Does it have any correlations to other features
&rarr; **Is the feature part of the model feature set? and why (one sentence)**

In [ ]:
last_tip_sequence.analyze_feature(orders_tip_features)

### Comparison of features

The comparison of the features is based on correlations and regression coefficients. The correlations and the 

## Model Training

Describe Pipeline Architecture. Time serious cross validation 


## Accuracy Prediction